## Setup

In [ ]:
def getBreakthroughPoint(df, col1, col2, patient_days, fill_method="fb"):
    '''
    :param df: dataframe (including col1, col2)
    :param col1: obj
    :param col2: obj moving average
    :param patient_days: patient days detected as breakthrough point
    :return: signal series
    '''
    sigPrice = []
    flag = -1  # A flag for the trend upward/downward

    for i in range(0, len(df)):
        if df[col1][i] > df[col2][i] and flag != 1:
            tmp = df['Close'][i:(i + patient_days + 1)]
            if len(tmp) == 1:
                sigPrice.append("buy")
                flag = 1
            else:
                if (tmp.iloc[1:] > tmp.iloc[0]).all():
                    sigPrice.append("buy")
                    flag = 1
                else:
                    sigPrice.append(nan)
        elif df[col1][i] < df[col2][i] and flag != 0:
            tmp = df['Close'][i:(i + patient_days + 1)]
            if len(tmp) == 1:
                sigPrice.append("sell")
                flag = 0
            else:
                if (tmp.iloc[1:] < tmp.iloc[0]).all():
                    sigPrice.append("sell")
                    flag = 0
                else:
                    sigPrice.append(nan)
        else:
            sigPrice.append(nan)

    sigPrice = series(sigPrice)
    for idx, value in enumerate(sigPrice):
        if not isna(value):
            if value == "buy":
                sigPrice.iloc[1:idx] = "sell"
            else:
                sigPrice.iloc[1:idx] = "buy"
            break
    # if fill_method == "bf":
    #
    # elif fill_method == ""
    sigPrice.ffill(inplace=True)
    return sigPrice
def stochastic(df, n=14, m=5, t=5):
    #데이터 프레임으로 받아오기 때문에 불필요

    #n 일중 최저가
    ndays_high = df['High'].rolling(window=n, min_periods=n).max()
    ndays_low = df['Low'].rolling(window=n, min_periods=n).min()
    fast_k = ((df['Close'] - ndays_low) / (ndays_high - ndays_low) * 100)
    slow_k = fast_k.ewm(span=m, min_periods=m).mean()
    slow_d = slow_k.ewm(span=t, min_periods=t).mean()
    df = df.assign(fast_k=fast_k, fast_d=slow_k, slow_k=slow_k, slow_d=slow_d)
    return df

## Loading data & Preprocessing

In [ ]:
# ===== raw data loading =====
# 한 종목코드에 대한 주가 정보를 로드

# 임의 선별
# 삼성전자
# NAVER
# 카카오

# 랜덤 선별
# rnd.seed(48)
# stock_list.iloc[rnd.randint(len(stock_list))]
# 금호석유
# 티움바이오
# 테크윙
# 제테마
# 주성엔지니어링
# 고바이오랩
# 고영

# Get Stock List
path = 'projects/dacon_stockprediction/open_week4/'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission_week4.csv'

# raw features (5개)
# 주가, 거래량, 기관순매수, 외인순매수, 뉴스 기사(embedding)

# derived features (14개)
# 주가이평, 거래량이평, 기관순매수이평, 외인순매수이평, 뉴스 기사에 대한 긍부정점수, 요일, sin변환(5일), cos변환(5일)
# 산식 보조 지표
# 1. 주가 관련 지표 : Stochastic(20), RSI(20), 볼린저밴드(20)
# 2. 거래량 관련 지표 : OBV, VR(20)
# 3. 혼합지표 : MFI(주가 + 거래량)

# 종목 코드 로드
stock_list = read_csv(os.path.join(path, list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x: str(x).zfill(6))

# Get Data & Modeling
# 분석할 date 변수 지정
start_date = '20201201'
end_date = '20211001'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
business_days = pd.DataFrame(pd.date_range(start_date, end_date, freq='B'), columns=['Date'])

# 선택 종목
stock_list.set_index("종목명", inplace=True)
selected_codes = ["삼성전자", "NAVER", "카카오", "금호석유", "티움바이오", "테크윙", "제테마", "주성엔지니어링", "고바이오랩", "고영"]
stock_list = stock_list.loc[selected_codes]["종목코드"]

# # 모든 종목
# stock_list.set_index("종목명", inplace=True)
# selected_codes = stock_list.index.tolist()
# stock_list = stock_list.loc[selected_codes]["종목코드"]

stock_dic = dict.fromkeys(selected_codes)
error_list = []
corr_list = []
anova_weekday = 0
anova_weeknum = 0
timeunit_gap = 1
metric_days = 14

# ==== selected feature =====
selected_features = ["date", "close", "fast_d", "obv", "fore_mv20", "inst_mv20", "kospi", "trading_amount_mv20"]
# selected_features = []

for stock_name, stock_code in stock_list.items():
    try:
        print("=====", stock_name, "=====")
        # 종목 주가 데이터 로드
        stock_dic[stock_name] = dict.fromkeys(["df", "target_list"])
        stock_df = stock.get_market_ohlcv_by_date(start_date, end_date, stock_code).reset_index()
        sleep(1)
        investor_df = stock.get_market_trading_volume_by_date(start_date, end_date, stock_code)[["기관합계", "외국인합계"]].reset_index()
        sleep(1)
        kospi_df = stock.get_index_ohlcv_by_date(start_date, end_date, "1001")[["종가"]].reset_index()
        sleep(1)

        stock_df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
        investor_df.columns = ["Date", "inst", "fore"]
        kospi_df.columns = ["Date", "kospi"]
        # 영업일과 주가 정보를 outer 조인
        train_x = pd.merge(business_days, stock_df, how='left', on="Date")
        train_x = pd.merge(train_x, investor_df, how='left', on="Date")
        train_x = pd.merge(train_x, kospi_df, how='left', on="Date")
        # 종가데이터에 생긴 na 값을 선형보간 및 정수로 반올림
        train_x.iloc[:,1:] = train_x.iloc[:,1:].ffill(axis=0).round(0)

        # ===== feature engineering =====
        # 요일 및 주차 파생변수 추가
        train_x['weekday'] = train_x["Date"].apply(lambda x: x.weekday())
        train_x['weeknum'] = train_x["Date"].apply(lambda x: week_of_month(x))
        cat_vars = ["weekday", "weeknum"]

        # 거래대금 파생변수 추가
        train_x['trading_amount'] = train_x["Close"] * train_x["Volume"]

        # 월별 주기성 특징을 잡기 위한 sin 및 cos 변환 파생변수 추가
        day_to_sec = 24 * 60 * 60
        month_to_sec = 20 * day_to_sec
        timestamp_s = train_x["Date"].apply(datetime.timestamp)
        timestamp_freq = round((timestamp_s / month_to_sec).diff(20)[20], 1)

        train_x['dayofmonth_freq_sin'] = np.sin((timestamp_s / month_to_sec) * ((2 * np.pi) / timestamp_freq))
        train_x['dayofmonth_freq_cos'] = np.cos((timestamp_s / month_to_sec) * ((2 * np.pi) / timestamp_freq))

        # OBV 파생변수 추가
        # 매수 신호: obv > obv_ema
        # 매도 신호: obv < obv_ema
        obv = [0]
        for i in range(1, len(train_x.Close)):
            if train_x.Close[i] > train_x.Close[i - 1]:
                obv.append(obv[-1] + train_x.Volume[i])
            elif train_x.Close[i] < train_x.Close[i - 1]:
                obv.append(obv[-1] - train_x.Volume[i])
            else:
                obv.append(obv[-1])
        train_x['obv'] = obv
        train_x['obv'][0] = nan
        train_x['obv_ema'] = train_x['obv'].ewm(com=metric_days, min_periods=metric_days).mean()


        # Stochastic 파생변수 추가
        # fast_d = moving average on fast_k
        train_x[["fast_k", "fast_d"]] = stochastic(train_x, n=metric_days)[["fast_k", "fast_d"]]


        # MFI 파생변수 추가
        # MFI = 100 - (100 / 1 + MFR)
        # MFR = 14일간의 양의 MF / 14일간의 음의 MF
        # MF = 거래량 * (당일고가 + 당일저가 + 당일종가) / 3
        # MF 컬럼 만들기
        train_x["mf"] = train_x["Volume"] * ((train_x["High"]+train_x["Low"]+train_x["Close"]) / 3)
        # 양의 MF와 음의 MF 표기 컬럼 만들기
        p_n = []
        for i in range(len(train_x['mf'])):
            if i == 0 :
                p_n.append(nan)
            else:
                if train_x['mf'][i] >= train_x['mf'][i-1]:
                    p_n.append('p')
                else:
                    p_n.append('n')
        train_x['p_n'] = p_n
        # 14일간 양의 MF/ 14일간 음의 MF 계산하여 컬럼 만들기
        mfr = []
        for i in range(len(train_x['mf'])):
            if i < metric_days-1:
                mfr.append(nan)
            else:
                train_x_=train_x.iloc[(i-metric_days+1):i]
                a = sum(train_x_['mf'][train_x['p_n']=='p']) / sum(train_x_['mf'][train_x['p_n'] == 'n'])
                mfr.append(a)
        train_x['mfr'] = mfr
        # 최종 MFI 컬럼 만들기
        train_x['mfi'] = 100 - (100/(1+train_x['mfr']))
        train_x["mfi_signal"] = train_x['mfi'].apply(lambda x: "buy" if x > 50 else "sell")

        # 이동평균 추가
        train_x["close_mv5"] = train_x["Close"].rolling(5, min_periods=5).mean()
        train_x["close_mv10"] = train_x["Close"].rolling(10, min_periods=10).mean()
        train_x["close_mv20"] = train_x["Close"].rolling(20, min_periods=20).mean()

        train_x["volume_mv5"] = train_x["Volume"].rolling(5, min_periods=5).mean()
        train_x["volume_mv10"] = train_x["Volume"].rolling(10, min_periods=10).mean()
        train_x["volume_mv20"] = train_x["Volume"].rolling(20, min_periods=20).mean()

        train_x["trading_amount_mv5"] = train_x["trading_amount"].rolling(5, min_periods=5).mean()
        train_x["trading_amount_mv10"] = train_x["trading_amount"].rolling(10, min_periods=10).mean()
        train_x["trading_amount_mv20"] = train_x["trading_amount"].rolling(20, min_periods=20).mean()

        train_x["inst_mv5"] = train_x["inst"].rolling(5, min_periods=5).mean()
        train_x["inst_mv10"] = train_x["inst"].rolling(10, min_periods=10).mean()
        train_x["inst_mv20"] = train_x["inst"].rolling(20, min_periods=20).mean()

        train_x["fore_mv5"] = train_x["fore"].rolling(5, min_periods=5).mean()
        train_x["fore_mv10"] = train_x["fore"].rolling(10, min_periods=10).mean()
        train_x["fore_mv20"] = train_x["fore"].rolling(20, min_periods=20).mean()

        train_x["kospi_mv5"] = train_x["kospi"].rolling(5, min_periods=5).mean()
        train_x["kospi_mv10"] = train_x["kospi"].rolling(10, min_periods=10).mean()
        train_x["kospi_mv20"] = train_x["kospi"].rolling(20, min_periods=20).mean()

        # 지표계산을 위해 쓰인 컬럼 drop
        train_x.drop(["mf", "p_n", "mfr", "Open", "High", "Low"], inplace=True, axis=1)

        # 2021/1/4 이후 일자만 선택
        train_x = train_x[train_x["Date"] >= datetime(2021, 1, 4)]
        train_x = train_x.dropna()
        train_x.reset_index(drop=True, inplace=True)

        # create target list
        target_list = []
        target_list.append(train_x["Close"])
        target_list.append(train_x["Close"].shift(-1))
        target_list.append(train_x["Close"].shift(-2))
        target_list.append(train_x["Close"].shift(-3))
        target_list.append(train_x["Close"].shift(-4))
        target_list.append(train_x["Close"].shift(-5))
        for idx, value in enumerate(target_list):
            value.name = "target_shift" + str(idx)

        # 컬럼이름 소문자 변환 및 정렬
        train_x.columns = train_x.columns.str.lower()
        train_x = pd.concat([train_x[["date"]], train_x.iloc[:,1:].sort_index(axis=1)], axis=1)

        # <visualization>
        # 시각화용 데이터프레임 생성
        train_bi = pd.concat([target_list[timeunit_gap], train_x], axis=1)[:-timeunit_gap]

        # 평균 상관관계를 측정하기 위해 연산
        corr_obj = train_bi.corr().round(3)
        corr_rows = corr_obj.index.tolist()
        corr_cols = corr_obj.columns.tolist()
        corr_list.append(corr_obj.to_numpy().round(3)[...,np.newaxis])

        # 상관관계 시각화
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.heatmap(corr_obj, cmap="YlGnBu", linewidths=0.2, annot=True)
        plt.xticks(rotation=45)
        fig.subplots_adjust(left=0.15, bottom=0.2)
        plt.title('Correlation Visualization on ' + stock_name, fontsize=15, fontweight="bold", pad=15)
        plt.savefig("projects/dacon_stockprediction/graphs/" + stock_name + ".png", dpi=300)
        plt.close()


        # feature 와 target 간 시각화
        # ===== scatter plot on numerical feature =====
        for i in train_x.columns:
            if i == "date" or i in cat_vars:
                pass
            else:
                fig, ax = plt.subplots(figsize=(12, 6))
                graph = sns.regplot(x=train_bi[i], y=train_bi["target_shift" + str(timeunit_gap)], color="green",
                                    scatter_kws={'s': 15}, line_kws={"color": "orange"})
                graph.set_title(i + " on " + stock_name, fontsize=15, fontweight="bold", pad=15)
                plt.show()
                createFolder('projects/dacon_stockprediction/graphs/' + stock_name)
                plt.savefig('projects/dacon_stockprediction/graphs/' + stock_name + "/" + i +".png", dpi=300)
                plt.close()

        # feature 분포 시각화
        # ===== hist plot on numerical feature =====
        for i in train_x.columns:
            if i == "date" or i in cat_vars:
                pass
            else:
                plt.figure(figsize=(12, 6))
                graph = sns.histplot(x=train_bi[i], bins=50, color="orange")
                graph.set_title("Distribution on " + stock_name + " (skewness : " + str(train_bi[i].skew().round(3)) + ")", fontsize=15, fontweight="bold", pad=15)
                graph.set_xlabel(graph.get_xlabel(), fontsize=12, fontweight="bold", labelpad=15)
                graph.set_ylabel(graph.get_ylabel(), fontsize=12, fontweight="bold", labelpad=15)
                plt.show()
                createFolder('projects/dacon_stockprediction/graphs/' + stock_name)
                plt.savefig('projects/dacon_stockprediction/graphs/' + stock_name + "/dist_" + i +".png", dpi=300)
                plt.close()

        # <feature scaling>
        # close, fast_d, kospi, trading_amount_mv20 -> 로그 변환
        train_x[["close", "fast_d", "kospi", "trading_amount_mv20"]] = train_x[["close", "fast_d", "kospi", "trading_amount_mv20"]].apply(np.log1p)

        # scaling 후 재 시각화
        # ===== hist plot on numerical feature =====
        for i in ["close", "fast_d", "kospi", "trading_amount_mv20"]:
            plt.figure(figsize=(12, 6))
            graph = sns.histplot(x=train_x[i], bins=50, color="orange")
            graph.set_title("After log scaling distribution on " + stock_name + " (skewness : " + str(train_x[i].skew().round(3)) + ")", fontsize=15,
                            fontweight="bold", pad=15)
            graph.set_xlabel(graph.get_xlabel(), fontsize=12, fontweight="bold", labelpad=15)
            graph.set_ylabel(graph.get_ylabel(), fontsize=12, fontweight="bold", labelpad=15)
            plt.show()
            createFolder('projects/dacon_stockprediction/graphs/' + stock_name)
            plt.savefig('projects/dacon_stockprediction/graphs/' + stock_name + "/dist_logTrans_" + i + ".png", dpi=300)
            plt.close()

        # categorical 변수에 대한 분산분석 (target 과의 상관관계 파악)
        # 귀무가설(H0) : 두 변수는 상관관계가 없다
        # 대립가설(H1) : 두 변수는 상관관계가 있다
        cat_list = train_bi.groupby("weekday")["target_shift" + str(timeunit_gap)].apply(list)
        anova_weekday += 1 / len(stock_list) if f_oneway(*cat_list)[1] <= 0.05 else 0

        cat_list = train_bi.groupby("weeknum")["target_shift" + str(timeunit_gap)].apply(list)
        anova_weeknum += 1 / len(stock_list) if f_oneway(*cat_list)[1] <= 0.05 else 0

        # export csv for BI tool
        corr_obj.to_csv("projects/dacon_stockprediction/bi_dataset/bi_corr_" + stock_name + ".csv", encoding="euc-kr", index_label=True, header=False)
        train_bi.to_csv("projects/dacon_stockprediction/bi_dataset/bi_data_" + stock_name + ".csv", encoding="euc-kr", index=False)

        # <feature selection>
        if len(selected_features) != 0:
            train_x = train_x[selected_features]
b 
        stock_dic[stock_name]["df"] = train_x.copy()
        stock_dic[stock_name]["target_list"] = target_list.copy()
    except:
        print("ERROR :", stock_name)
        error_list.append((stock_name, stock_code))
del train_x